## DIA 040: Integracion de Tensorflow Serving para Model Serving

Objetivo Principal:

Desacoplar la lógica de predicción del modelo de la aplicación Flask mediante TensorFlow Serving, permitiendo actualizar y escalar el modelo de forma independiente.
Implementación del Endpoint /tf_predict:

Se añadió un endpoint protegido con JWT que recibe una imagen, la preprocesa (conversión a escala de grises, redimensionamiento a 28x28 y normalización) y envía la solicitud al servicio TensorFlow Serving.
Se utiliza la librería requests para enviar un POST a TensorFlow Serving, esperando la respuesta del modelo.
La respuesta de TensorFlow Serving se retorna al cliente en formato JSON.
Beneficios Clave:

Desacoplamiento: Se separa el model serving de la lógica de la API, facilitando actualizaciones y escalabilidad.
Rendimiento: TensorFlow Serving está optimizado para servir modelos de ML, lo que mejora los tiempos de respuesta.
Gestión de Versiones: Permite desplegar y gestionar diferentes versiones del modelo sin interrumpir la operación de la API.
Código Completo (api.py)
python
Copiar
import os
import io
import numpy as np
import requests
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from PIL import Image
import logging

# Configuración básica de Flask y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de JWT
jwt = JWTManager(app)

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Endpoint de Login para obtener el token JWT (para pruebas)
@app.route('/login', methods=['POST'])
def login():
    """
    Endpoint para el login de usuarios.
    ---
    Request Body:
      - username: string
      - password: string
    Response:
      - access_token: string
    """
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    # En este ejemplo, asumimos que las credenciales son válidas
    access_token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

# Endpoint de Health Check
@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# Endpoint para realizar predicciones utilizando TensorFlow Serving
@app.route('/tf_predict', methods=['POST'])
@jwt_required()
def tf_predict():
    """
    Endpoint para predecir utilizando TensorFlow Serving.
    Se espera un archivo de imagen en formato multipart/form-data.
    ---
    Request:
      - file: imagen (form-data)
    Response:
      - JSON con la predicción del modelo
    """
    if 'file' not in request.files:
        return jsonify({"error": "No file part in the request"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected for uploading"}), 400

    try:
        # Preprocesamiento de la imagen:
        # 1. Convertir a escala de grises.
        # 2. Redimensionar a 28x28 (tamaño típico de MNIST).
        # 3. Normalizar los valores a [0, 1].
        image = Image.open(file.stream).convert('L')
        image = image.resize((28, 28))
        image_array = np.array(image).astype('float32') / 255.0
        
        # Expandir dimensiones para ajustar la forma requerida por el modelo: (1, 28, 28, 1)
        image_array = np.expand_dims(image_array, axis=0)   # De (28, 28) a (1, 28, 28)
        image_array = np.expand_dims(image_array, axis=-1)    # De (1, 28, 28) a (1, 28, 28, 1)

        # Crear el payload para TensorFlow Serving
        payload = {"instances": image_array.tolist()}
        logger.info("Enviando solicitud a TensorFlow Serving con payload: %s", payload)

        # URL del servicio TensorFlow Serving
        tf_serving_url = os.getenv("TF_SERVING_URL", "http://localhost:8501/v1/models/model:predict")
        response = requests.post(tf_serving_url, json=payload)

        if response.status_code != 200:
            logger.error("Error de TensorFlow Serving: %s", response.text)
            return jsonify({"error": "Error from TensorFlow Serving", "details": response.text}), 500

        prediction = response.json()
        logger.info("Predicción recibida: %s", prediction)
        return jsonify(prediction), 200

    except Exception as e:
        logger.exception("Excepción durante la predicción")
        return jsonify({"error": str(e)}), 500

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run(debug=True)
Explicación del Código
Configuración y Extensiones:

Se configura la aplicación Flask, se establece el secreto para JWT y se inicializa el módulo JWT.
Se configura el logging para capturar información sobre el proceso de predicción y cualquier error que pueda ocurrir.
Endpoint /login:

Permite a los usuarios iniciar sesión y obtener un token JWT, el cual se utiliza para autenticar solicitudes a otros endpoints.
Endpoint /tf_predict:

Protegido con @jwt_required(), este endpoint recibe un archivo de imagen.
La imagen se procesa: se convierte a escala de grises, se redimensiona a 28x28, se normaliza, y se ajusta su forma para que coincida con la entrada esperada del modelo (formato MNIST).
Se construye un payload JSON y se envía a la URL de TensorFlow Serving utilizando requests.post().
La respuesta del servicio se devuelve al cliente en formato JSON.
Manejo de Errores:

Se verifican posibles errores durante el preprocesamiento de la imagen y la comunicación con TensorFlow Serving.
Se registran mensajes de error utilizando el logger para facilitar la depuración.
Endpoint /health:

Permite verificar rápidamente que la aplicación se esté ejecutando correctamente.